# Тестирование модели (загруженной локально)

## 1. Загрузка и использование с **AutoTokenizer** и **AutoModelForSequenceClassification**

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

d:\mycode\classifier-emotions-text-ru\.venv-classifier\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" >> Используем устройство: {device}")

 >> Используем устройство: cuda


In [20]:
model_name = "cointegrated/rubert-tiny2-cedr-emotion-detection"
model_path= "../models/rubert-tiny2-cedr-emotion-detection/"

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    use_fast=True,   # Используем быстрый токенизатор
    truncation=True, # Обрезаем длинные тексты
    max_length=512,  # Максимальная длина (у RuBERT обычно 512)
    return_tensors="pt"  # Возвращаем тензоры PyTorch
)

model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()
print()

In [4]:
labels = model.config.id2label
labels

{0: 'no_emotion', 1: 'joy', 2: 'sadness', 3: 'surprise', 4: 'fear', 5: 'anger'}

In [5]:
text = "Я очень рад, что ты вернулся!"

inputs = tokenizer(
    text,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

inputs = inputs.to(device)

model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)

id2label = model.config.id2label

predicted_class_id = torch.argmax(probs, dim=-1).item()
predicted_label = id2label[predicted_class_id]
confidence = probs[0][predicted_class_id].item()
predicted_label, confidence

('joy', 0.9986925721168518)

## 2. Загрузка и использование с **transformers.pipeline**

In [6]:
import torch
from transformers import pipeline

In [7]:
classifier = pipeline(
    "text-classification", 
    model=model_path,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

text = "Я очень рад, что ты вернулся!"

with torch.no_grad():
    result = classifier(text)
    print(result)

Device set to use cuda


[{'label': 'joy', 'score': 0.9798190593719482}]


## 3. Обращение при помощи **InferenceClient** (*HF Inference API*)

In [28]:
import os
from huggingface_hub import InferenceClient

In [22]:
client = InferenceClient(
    model=model_name,
    provider="hf-inference",
    api_key=os.environ["HF_TOKEN"],
)
client

<InferenceClient(model='cointegrated/rubert-tiny2-cedr-emotion-detection', timeout=None)>

In [23]:
text = 'Я очень рад, что ты вернулся!'

result = client.text_classification(
    text=text
)

result

[TextClassificationOutputElement(label='joy', score=0.9798190593719482),
 TextClassificationOutputElement(label='sadness', score=0.023603854700922966),
 TextClassificationOutputElement(label='no_emotion', score=0.011824214830994606),
 TextClassificationOutputElement(label='surprise', score=0.010615072213113308),
 TextClassificationOutputElement(label='anger', score=0.008691806346178055)]

**client.text_classification**

- **Returns**:
    * `List[TextClassificationOutputElement]`: a list of [`TextClassificationOutputElement`]
    items containing the predicted label and associated probability.

- **Raises**:
    * [`InferenceTimeoutError`]:
        If the model is unavailable or the request times out.
    * `HTTPError`:
        If the request fails with an HTTP error status code other than HTTP 503.